In [11]:
from pathlib import Path
import subprocess
import os
from typing import Iterator, Set, Tuple

def find_git_repositories(directory: Path) -> Iterator[Path]:
  for root, dirs, files in os.walk(directory):
    for dir in dirs:
      if (Path(root) / dir / ".git").exists():
        yield Path(root) / dir

def check_git_status(repository: Path) -> Tuple[Set[str], Set[str]]:
  status = subprocess.check_output(["git", "-C", str(repository), "status"]).decode("utf-8")

  untracked_files = set()
  uncommitted_files = set()
  for line in status.splitlines():
    if line.startswith("??"):
      untracked_files.add(line[3:])
    elif not line.startswith("??") and line:
      uncommitted_files.add(line[3:])

  return untracked_files, uncommitted_files

def main():
  directory = Path(".")

  for repository in find_git_repositories(directory):
    unversioned_files, uncommitted_files = check_git_status(repository)

    if unversioned_files or uncommitted_files:
      for file in unversioned_files:
        print(f"Untracked: {repository}/{file}")
      for file in uncommitted_files:
        print(f"Uncommitted: {repository}/{file}")

if __name__ == "__main__":
  main()
